5. Predict congressional speech --- it is in opt/cong (or you can download via capitolwords API based on the script you wrote, we want to predict for each congress.)
   Predict each category. If you have 9 categories, you will get 9 columns. 

In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
NTH = 111

## Read Congressional speech dataset

In [3]:
#df = pd.read_csv('../cong/cong_107_reg.csv', nrows=10)
df = pd.read_csv('../cong/capitolwords_clean_%d_dw.csv' % NTH, nrows=10)
df.columns

Index([u'biocong', u'speaker_state', u'speaker_raw', u'speaker_first',
       u'congress', u'title', u'origin_url', u'number', u'id', u'volume',
       u'chamber', u'session', u'speaker_last', u'pages', u'speaker_party',
       u'date', u'bills', u'bioguide_id', u'order', u'speaking',
       u'capitolwords_url', u'bills1', u'bills1n', u'chamb', u'bills_unique1',
       u'major', u'minor', u'billnum', u'idno', u'cong', u'state', u'cd',
       u'statenm', u'party', u'name', u'dwnom1', u'dwnom2', u'bootse1',
       u'bootse2', u'corr12', u'logl', u'nchoices', u'errors', u'gmp',
       u'fullname', u'bioguide_id_1'],
      dtype='object')

In [4]:
df = pd.read_csv('../cong/capitolwords_clean_%d_dw.csv' % NTH, usecols=['party', 'date', 'speaking'])
df.columns = ['date', 'text', 'party']
df

,date,text,party
0,2009-05-06,mr bradi thank yield mr speaker repres hirono ...,100
1,2010-02-23,thank mr chairman thank doc regard member cour...,100
2,2009-02-04,madam speaker rise today introduc nativ hawaii...,100
3,2009-10-01,madam speaker regret miss rollcal vote no pres...,100
4,2009-07-15,mr chair rise today oppos amend offer repres f...,100
5,2010-02-23,mr speaker would like enter record letter favo...,100
6,2010-02-23,could take leav floor mr speaker without menti...,100
7,2010-02-23,gentleman washington privileg close,100
8,2010-02-23,want speak one time final speaker,100
9,2009-03-19,madam speaker regret miss rollcal vote present...,100


In [5]:
import re
import textwrap

def insert_chars_split_marker(text, cc=2500):
    # FIXME: text still has number
    text = re.sub('\d+', '', text)
    out = '|'.join(textwrap.wrap(text, cc))
    return out

def insert_words_split_marker(text, wc=500):
    text = re.sub('\d+', '', text)
    words = text.split()
    out = ''
    for i, w in enumerate(words):
        if i != 0 and i % wc == 0:
            out += '|' + w
        else:
            out += ' ' + w
    return out

In [6]:
if False:
    df['text'] = df['text'].astype(str).apply(lambda c: insert_chars_split_marker(c, 2500))

In [7]:
if False:
    s = df['text'].str.split('|', expand=True).stack()
    i = s.index.get_level_values(0)
    new_df = df.loc[i].copy()
    new_df['chunk'] = s.index.get_level_values(1)
    new_df['text'] = s.values
    df = new_df.reset_index(drop=True)

In [8]:
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfTransformer

## Major Model

In [9]:
vect_name = '../models/vec_count_bills_23gram_new.joblib'
clf_name = '../models/major_bills_clf_liblinear_new.joblib'

In [10]:
vect = joblib.load(vect_name)
clf = joblib.load(clf_name)
len(vect.vocabulary_)

686467

In [11]:
X = vect.transform(df.text.astype(str))
tfidf = TfidfTransformer()
X = tfidf.fit_transform(X)
y_pred = clf.predict(X)

In [12]:
if False:
    conf = clf.decision_function(X)
    conf_df = pd.DataFrame(conf)

In [13]:
if False:
    topics_df = pd.read_csv('../data/topic_code.csv')
    columns = []
    for c in clf.classes_:
        cname = topics_df[topics_df.code == c].topic.values[0]
        columns.append(cname)
    conf_df.columns = columns
    result = pd.concat([df[['date', 'party', 'text']], conf_df], axis=1)
    result

In [14]:
df['topic'] = y_pred
df['cong_number'] = NTH
df

,date,text,party,topic,cong_number
0,2009-05-06,mr bradi thank yield mr speaker repres hirono ...,100,20,111
1,2010-02-23,thank mr chairman thank doc regard member cour...,100,21,111
2,2009-02-04,madam speaker rise today introduc nativ hawaii...,100,21,111
3,2009-10-01,madam speaker regret miss rollcal vote no pres...,100,3,111
4,2009-07-15,mr chair rise today oppos amend offer repres f...,100,20,111
5,2010-02-23,mr speaker would like enter record letter favo...,100,21,111
6,2010-02-23,could take leav floor mr speaker without menti...,100,21,111
7,2010-02-23,gentleman washington privileg close,100,3,111
8,2010-02-23,want speak one time final speaker,100,15,111
9,2009-03-19,madam speaker regret miss rollcal vote present...,100,3,111


In [15]:
df.to_csv('../data/congress_%d_predict_major_label.csv' % NTH, index=False)